<h1>Korpuse loomine</h1>
<h5>Olemasoleva andmestiku puhastamine ning korpuse loomine</h5>

In [2]:
import json
import os, os.path

<h5>Loeme sisse postitused ja kommentaarid</h5>

In [141]:
posts = []
with open("output.json", "r", encoding = 'utf-8') as file:
    for line in file:
        rida = json.loads(line)
        posts.append(rida)

In [149]:
comments = []
with open("comments_output.json", "r", encoding = 'utf-8') as file:
    for line in file:
        rida = json.loads(line)
        comments.append(rida)

<h5>Struktureerime kommentaarid ning lisame need postituste hulka ID järgi.</h5>

In [150]:
comments_copy = comments.copy()
#print(comments[0])
#print(comments[0]["link_id"])
dict_comments = dict()
for comment in comments_copy:
    #print(comment)
    body = comment["body"]
    comment.pop('body', None)
    link_id = comment["link_id"].split("_")[1]
    if link_id not in dict_comments.keys():
        dict_comments[link_id] = {comment["id"]:{
            "text":body,
            "meta":comment
        }
        }
        
    else:
        dict_comments[link_id][comment["id"]] = {
        "text": body,
        "meta": comment
    }

In [153]:
posts_copy = posts.copy()
title_string = '''data/Eesti_'''

#print(posts[0]["selftext"])
#print(posts[0]["created_utc"][:10])
keys = dict_comments.keys()

for post in posts_copy:
    #print(post)
    if "selftext" in post:
        text = post["selftext"]
        date = post["created_utc"].replace(" ", "_")
        date = date.replace(":", ".")
        post.pop('selftext', None)
    else:
        text = ""
    
    comments = []
    if post["id"] in keys:
        comments = dict_comments[post["id"]]
    
    data = {"text":text,
            "meta":post,
            "comments":comments
            }
    
    title = title_string + date + ".json"
    with open(title, 'w') as f:
        json.dump(data, f)

# kui postitusel on sisu tekst, siis on tegemist tekstipostitusega
# kui postitusel on olemas muutuja "poll_data" on tegemist polliga
# kui on tegemist link tüüpi postitusega, siis on url väljal mingi redditist erinev url millele viidatakse
# kui on tegemist pildiga, siis on url väljal näha pildiformaadi laiendit